# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 13** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Integrantes**:
- Lorena Ruelas Gaytán
- Yael Alejandro Rodríguez Barreto
- Ximena Isaac Horta
- Alberto Renteria Camacho

In [6]:
import findspark
findspark.init()

#### Spark Conexion

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-K-means") \
    .master("spark://9c456350d25c:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

#### Data preparation

In [8]:
from team_name.spark_utils import SparkUtils

csv_path = "/home/jovyan/notebooks/data/wines_data/wine-clustering.csv"

headers = [ "Alcohol", "Malic_Acid", "Ash", "Ash_Alcanity",
        "Magnesium", "Total_Phenols", "Flavanoids", "Nonflavanoid_Phenols",
        "Proanthocyanins", "Color_Intensity", "Hue", "OD280", "Proline"
]

schema = SparkUtils.generate_schema([(head, "float") for head in headers])

# Convert list to a DataFrame
wines_df = spark.read \
        .schema(schema) \
        .option("header", "true") \
        .csv(csv_path)

wines_df = wines_df.dropna(how="any")
wines_df.show(10)

+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+
|  14.23|      1.71|2.43|        15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04| 3.92| 1065.0|
|   13.2|      1.78|2.14|        11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|  3.4| 1050.0|
|  13.16|      2.36|2.67|        18.6|    101.0|          2.8|      3.24|                 0.3|           2.81|           5.68|1.03| 3.17| 1185.0|
|  14.37|      1.95| 2.5|        16.8|    113.0|         3.85|      3.49|                0.24|           2.18|            7.

### Assemble the features into a single vector column

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=headers, outputCol="features")
assembled_df = assembler.transform(wines_df)

---
# KMeans at the same time

#### Initialize KMeans

In [ ]:
from pyspark.ml.clustering import KMeans

k_values = [2, 10, 15, 20]
kmeans = [KMeans().setK(2).setSeed(19) for k in k_values]

#### Training

In [ ]:
models = [k.fit(assembled_df) for k in kmeans]

#### Predictions

In [ ]:
predictions = [model.transform(assembled_df) for model in models]

#### Evaluate model

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print(f"Silhouette score: {silhouette}")

# Show the result
print("Cluster Centers: ")
for center in model.clusterCenters():
    print(center)

In [ ]:
sc.stop()